In [4]:
#Extra stuff imported, I will keep it like this for testing 
import os
import re
import numpy as np
import pathlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.metrics import CategoricalAccuracy
from sklearn.metrics import confusion_matrix
import seaborn as sns


IMG_SRC = pathlib.Path.cwd() / 'images'

In [9]:



def make_model(n, n_classes):

    model = Sequential([
        Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),

        Conv2D(32, (5, 64*n)),
        LeakyReLU(alpha=0.1),
        MaxPooling2D(pool_size=(2*n,2*n)),
        Dropout(0.25),

        Conv2D(32, (15, 64)),
        LeakyReLU(alpha=0.1),
        MaxPooling2D(pool_size=(5,5)),
        Dropout(0.25),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.1),
        
        Dense(n_classes, activation='softmax')
    ])

    return model


In [15]:
def train_overnight(data_path, model_name, n_epochs, n):
    train_path = data_path / 'train'
    test_path = data_path / 'test'

    # Data Preprocessing using ImageDataGenerator
    train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

    # Load train and validation sets (90% for training, 10% for validation)
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=IMG_SIZE,
        color_mode="rgb",
        class_mode="categorical",
        batch_size=BATCH_SIZE,
        subset="training",
        shuffle=True,
        seed=RANDOM_STATE
    )

    validation_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=IMG_SIZE,
        color_mode="rgb",
        class_mode="categorical",
        batch_size=BATCH_SIZE,
        subset="validation",
        shuffle=True,
        seed=RANDOM_STATE
    )

    test_generator = train_datagen.flow_from_directory(
        directory=test_path,
        target_size=IMG_SIZE,
        color_mode="rgb",
        batch_size=1,
        class_mode=None,
        shuffle=False,
        seed=RANDOM_STATE
    )

    model = make_model(n, len(train_generator.class_indices))
    model.compile(optimizer=Adam(learning_rate=0.0001),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])
    
    
    history = model.fit(
        train_generator,
        epochs=n_epochs,
        validation_data=validation_generator
    )

    model.save(f'weights/{model_name}.keras')
    print(f"Model saved as '{model_name}.keras'.")

    score = model.evaluate(validation_generator)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.savefig(f'figures/{model_name}_loss_accuracy.png')
    plt.close()


        
    # Get true labels and predicted labels
    y_true = test_generator.classes
    class_names = list(test_generator.class_indices.keys())

    # Predict probabilities
    # the automatic shuffling needs to be disabled
    # so that the indices in y_true and y_pred refer to the same images  
    # test_generator.shuffle = False
    # test_generator.index_array = None

    y_pred_probs = model.predict(test_generator, verbose=1)
    y_pred = np.argmax(y_pred_probs, axis=1)

    # Generate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cm_percent = np.nan_to_num(cm_percent)  # Handle NaN if any class has 0 samples

    # Plot
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_percent, annot=True, cmap="viridis", xticklabels=class_names, yticklabels=class_names, cbar_kws={'label': 'Proportion'})
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Normalized Confusion Matrix (Train Set)")
    plt.tight_layout()
    plt.savefig(f'figures/{model_name}_cm.png')
    plt.close()

In [16]:

# Parameters
IMG_SIZE = (1000, 512)       # time, n_subcarriers
BATCH_SIZE = 16
RANDOM_STATE = 42

train_overnight(IMG_SRC / 'Tuto_Movement', 'tuto_movement', 50, 2)

# -------------------------------------------------------------------------------------------

# Parameters
IMG_SIZE = (1000, 512)       # time, n_subcarriers
BATCH_SIZE = 16
RANDOM_STATE = 42

train_overnight(IMG_SRC / 'Tuto_Person', 'tuto_person', 50, 2)

# -------------------------------------------------------------------------------------------

# Parameters
IMG_SIZE = (1000, 512)       # time, n_subcarriers
BATCH_SIZE = 16
RANDOM_STATE = 42

train_overnight(IMG_SRC / 'Movement', 'lab_movement', 50, 2)

# -------------------------------------------------------------------------------------------

# Parameters
IMG_SIZE = (1000, 512)       # time, n_subcarriers
BATCH_SIZE = 16
RANDOM_STATE = 42

train_overnight(IMG_SRC / 'Person', 'lab_person', 50, 2)

Found 408 images belonging to 3 classes.
Found 44 images belonging to 3 classes.
Found 74 images belonging to 3 classes.
Epoch 1/50
26/26 [==============================] - 134s 5s/step - loss: 0.9862 - accuracy: 0.4877 - val_loss: 0.9555 - val_accuracy: 0.3636
Epoch 2/50
26/26 [==============================] - 58s 2s/step - loss: 0.9061 - accuracy: 0.4828 - val_loss: 0.9201 - val_accuracy: 0.5000
Epoch 3/50
26/26 [==============================] - 58s 2s/step - loss: 0.8542 - accuracy: 0.5466 - val_loss: 0.8354 - val_accuracy: 0.3864
Epoch 4/50
26/26 [==============================] - 58s 2s/step - loss: 0.7198 - accuracy: 0.6569 - val_loss: 0.8630 - val_accuracy: 0.3409
Epoch 5/50
26/26 [==============================] - 58s 2s/step - loss: 0.6168 - accuracy: 0.7108 - val_loss: 0.9973 - val_accuracy: 0.4773
Epoch 6/50
26/26 [==============================] - 58s 2s/step - loss: 0.5284 - accuracy: 0.7500 - val_loss: 0.8237 - val_accuracy: 0.5909
Epoch 7/50
26/26 [====================